# Lesson 6.1: Prompt Engineering

**Note:** In this lesson you will be given examples for creating prompts based on using data and pandas as well as non-data examples. You will be shown how to structure your prompts for both types of examples.

## <ins>Setup</ins>  

**Reminder:** Don't forget to setup your virtual environment, choose your python interpreter and install the ipykernel.

### 1. Retrieving Your API Key

Before we begin, you will need to retrieve your API key from Gemini.

Use the following set of instructions to sign up for an account and retrieve your API key.

[Gemini API Key](https://github.com/jdrichards-pursuit/gemini-api-key-acquire?tab=readme-ov-file)

### 2. Setting Up Your Environment Variables

Now that you have your API key, you can set up your environment variables.

Create a new file called `.env` and add the following line of code:

```bash
API_KEY=<your-api-key>
```

### 3. Installing Required Libraries

Now that you have your API key, you can install the required libraries.

```bash
pip install python-dotenv
```

### 4. Importing Required Libraries

Now that you have your API key, you can import the required libraries. When you run this code, it will print your API key to the console.




In [ ]:
from dotenv import load_dotenv
import os

load_dotenv()

api_key = os.getenv("API_KEY")

print(api_key)

After viewing your API key, you may now want to remove the print statement for security reasons.

### 5. Importing Required Libraries

For this lesson you will also need to install the following libraries:

```bash
pip install google-generativeai pandas
```

### 6. Import Packages

Next, import these packages into your environment.


In [ ]:
import google.generativeai as genai
import pandas as pd

### 7. Set Up API Key

You will need to set up your API key using your already created environment variable.


In [ ]:
os.environ['API_KEY'] = api_key
genai.configure(api_key=os.environ['API_KEY'])

## <ins>Prompt Process Under the Hood</ins>

Before we begin digging into prompt engineering, let's familiarize ourselves on a high level with what happens under the hood when we send a prompt. This is definitely 'above our paygrade' and not what we will be focusing on, but it is good to be aware of. Take a look at this link which describes [The Process of A Prompt](https://github.com/jdrichards-pursuit/prompt-process-explained).

### Parts of a Prompt

When creating a prompt using the Gemini API, there are three main parts to consider:

1. **Model**: The model you are using to generate the response. In our case, we are using the `gemini-1.5-flash` model.
2. **Parameters**: Parameters are additional settings or options that you can send to the model to guide the response. We will cover these in more detail later in the lesson.
3. **Prompt**: A prompt is the input or instruction given to an AI model to elicit a specific response or output. It's the text that guides the AI in understanding what kind of information or task you're requesting.


### Model
For this lesson, we will be using the `gemini-1.5-flash` model because it is free and '...is a fast and versatile multimodal model for scaling across diverse tasks.' to quote the [Gemini API Documentation](https://ai.google.dev/gemini-api/docs/models/gemini#gemini-1.5-flash).




### Parameters

When we send a prompt to the model, we can also send parameters to the model. These parameters are used to guide the model's response. In Gemini API, there are several parameters we can send to the model, but for now we will focus on the following:

1. `temperature`: This parameter controls the randomness of the model's response. A temperature of 0 is deterministic, meaning the model will always return the same response for a given prompt. A temperature between 0 and 1 will make the response more random. 
2. `max_output_tokens`: This parameter controls the maximum number of tokens in the model's response.
3. `num_return_sequences`: This parameter controls the number of alternative completions to generate.
4. `min_length`: This parameter controls the minimum number of tokens in the model's response.
5. `max_length`: This parameter controls the maximum number of tokens in the model's response.


There are many more parameters, but we will not focus on those for now.

#### 1. Temperature

- This parameter controls the randomness of the generated text.

- Think of it like adjusting the "creativity" or "originality" of the generated text:

    - A low temperature setting is ideal for:

        - Answering factual questions (e.g., "What is the capital of France?")
        - Generating technical documentation or instruction manuals
        - Providing definitions or explanations of complex terms
        - Summarizing scientific papers or reports


    - A high temperature setting is beneficial for:

        - Generating story ideas or plot twists
        - Coming up with unique marketing slogans or taglines
        - Brainstorming solutions to open-ended problems


- By default, the API uses a temperature of 0.5, which is a good balance between creativity and coherence.

#### 2. Max Output Tokens

- This parameter sets the maximum number of tokens in the generated text.
- A token is the smallest unit of text that the model can generate.
- If you set max_output_tokens to 20, the API will generate text that's around 20 tokens long. 
- **Note:** Tokens are not the same as characters or words. They can be a word, a phrase, a sentence, etc.


#### 3. Num Return Sequences

- This parameter sets the number of alternative completions to generate.
- If you set num_return_sequences to 3, the API will generate 3 different completions for the same prompt.


#### 4. Min Length

- This parameter sets the minimum number of characters (letters, spaces, punctuation) in the generated text.
- Think of it like setting a word count for a minimum number of sentences. You might ask someone to write at least 5 sentences.
- The API will generate text that's as close to the minimum length as possible, but it might not reach the exact limit.
- If you set min_length to 20, the API will generate text that's around 20 characters long.
- There is no default value for this parameter, so you will always need to set it.

#### 5. Max Length

- This parameter sets the maximum number of characters (letters, spaces, punctuation) in the generated text.
- Think of it like setting a word count for a maximum number of sentences. You might ask someone to write at most 5 sentences.
- The API will generate text that's as close to the maximum length as possible, but it might not reach the exact limit.
- If you set max_length to 20, the API will generate text that's around 20 characters long.
- There is no default value for this parameter, so you will always need to set it.

### Default Parameters

You do not have to set all of these parameters and we will not explain them all,, but it is good to be aware of them. When they are not specified, the API will use the default values. The defaults are:

```python
generation_config = {
    "model": "gemini-1.5-flash",
    "temperature": 0.7,
    "max_output_tokens": 2048,
    "num_return_sequences": 1,
    "top_p": 1,
    "top_k": 32,
    "stop_sequences": [],
    "candidate_count": 1,
    "safety_settings": [
        {
            "category": "HARM_CATEGORY_HARASSMENT",
            "threshold": "BLOCK_MEDIUM_AND_ABOVE"
        },
        {
            "category": "HARM_CATEGORY_HATE_SPEECH",
            "threshold": "BLOCK_MEDIUM_AND_ABOVE"
        },
        {
            "category": "HARM_CATEGORY_SEXUALLY_EXPLICIT",
            "threshold": "BLOCK_MEDIUM_AND_ABOVE"
        },
        {
            "category": "HARM_CATEGORY_DANGEROUS_CONTENT",
            "threshold": "BLOCK_MEDIUM_AND_ABOVE"
        }
    ],
    "prompt_feedback": {
        "safety_ratings": []
    }
}
```

### Lets create a function, `get_completion`, that will submit a prompt to the Gemini API and return the response:

In [ ]:
def get_completion(prompt, model="gemini-1.5-flash", **kwargs):
    model = genai.GenerativeModel(model)
    
    # Create a generation_config dictionary with default values
    generation_config = {
        "temperature": 0.8,
        "min_length": 20,
        "max_length": 200,
    }
    
    # Update generation_config with any provided kwargs
    generation_config.update(kwargs)
    
    response = model.generate_content(prompt, generation_config=generation_config)
    return response.text

#### Walkthrough of the Code

1. We first create a function called `get_completion` that takes in a prompt and any number of keyword arguments called `kwargs`.
    - `**kwargs` allows us to pass in any number of keyword arguments to the function.
    - the double asterisk is used to unpack the keyword arguments into a dictionary. In JavaScript, this would be `{...args}`.

2. We then create a `generation_config` dictionary with default values for the parameters we want to default to.

3. We then update the `generation_config` dictionary with any provided keyword arguments. This allows us to override the default values when we call the function.

4. We then generate the response from the model using the `generate_content` method on the model dictionary.

5. Finally, we return the response.

### Let's set the `get_completion` function call value to a variable, response, and print the response:

In [ ]:
response = get_completion(
    prompt,
    temperature=0.9,
    max_output_tokens=1000
)

print(response)

## <ins>Prompt Engineering Techniques</ins>

Prompt engineering is a crucial skill for effectively leveraging AI language models like Gemini Api, ChatGPT, Claude, etc. It involves crafting inputs that guide the AI to produce desired outputs. Mastering these techniques allows users to get more accurate, relevant, and useful responses from AI systems.

Key prompt engineering techniques include:
1. Formulating clear and specific questions
2. Providing relevant context
3. Specifying desired output format
4. Including examples to anchor prompts
5. Breaking down complex tasks into steps
6. Encouraging multiple alternatives
7. Iterative prompting and refinement

These techniques help improve the AI's reasoning capabilities, enable more structured outputs, and enhance overall interaction quality with language models.

### 1. Formulating Effective Questions

Whether you're analyzing data or seeking general information, asking clear and specific questions is crucial. Let's explore how to frame your queries effectively in both data-related and general contexts.

#### Clear vs. Vague Instructions

#### Data Example:
```python
# Good: Clear and specific question
"""
What is the average sale price of houses with more than 3 bedrooms 
in the 'Seattle' area from our housing dataset?
"""

# Bad: Vague and ambiguous question
"""
What can you tell me about the houses in the dataset?
"""
```

#### Non-Data Example:
```python
# Good: Clear and specific question
"""
What are the three main causes of deforestation in the Amazon rainforest 
over the past decade?
"""

# Bad: Vague and ambiguous question
"""
Tell me about forests.
"""
```

#### Discuss the difference between the clear and vague instructions listed above.

### Practice (10 minutes)

Imagine you're asking someone to explain the concept of climate change.

1. Write a vague instruction for explaining climate change.
2. Rewrite the instruction to make it clear and specific. Consider asking for a few key points to be covered, such as but not limited to:

    - Definition of climate change
    - Main causes (e.g., greenhouse gases, human activities)
    - Observable effects (e.g., rising temperatures, extreme weather events)
    Potential future impacts
    - Possible solutions or mitigation strategies
    - Request the use of simple language suitable for a general audience
    - Ask for one or two specific examples or statistics to illustrate key points

3. Show the class what you wrote for both prompts and explain the differences in the two model responses.

In [ ]:
# comment and uncomment your prompts one at a time to see the difference
# Your vague instruction:
prompt = """
"""
# Your clear and specific instruction:

prompt = """
"""

response = get_completion(
    prompt
)

print(response)

### 2. Providing Relevant Context

Providing relevant context is crucial for guiding the model to produce accurate and relevant responses. This can be achieved by including relevant information about the dataset or the task at hand.

##### Data Example:
```python
# Good: Includes relevant context
"""
Our dataset 'sales.csv' contains columns for 'price', 'bedrooms', and 'location'.
Can you show me how to calculate the median price for each bedroom category?
"""

# Bad: Lacks necessary context
"""
Calculate the median prices.
"""
```

##### Non-Data Example:
```python
# Good: Includes relevant context
"""
In the context of object-oriented programming in Python, 
can you explain the concept of inheritance with a simple example?
"""

# Bad: Lacks necessary context
"""
What is inheritance?
"""
```

#### Discuss the difference between the two prompts above. Why is one better than the other?


### Practice (10 minutes)

Imagine you're asking for advice on how to care for a specific type of houseplant.

1. Write a prompt without providing any context.
2. Then, rewrite the prompt to include relevant context.
3. Show the class what you wrote for both prompts and explain the differences in the two model responses.

Relevant context for houseplant care could include:
- Plant species or variety
- Plant size and age
- Light conditions
- Location in the home
- Temperature and humidity levels
- Current plant health
- Watering habits


In [ ]:
# comment and uncomment your prompts one at a time to see the difference
# Your prompt without context:
prompt = """
"""
# Your prompt with context:
prompt = """
"""

response = get_completion(
    prompt
)

print(response) # this is the response from the model


### 3. Specifying Constraints in Your Queries

When working with data or seeking information, it's important to specify any constraints or limitations in your request. This helps ensure you get precisely the information you need in the format you want.

#### Data Example:



```python
# Good: Specifies constraints clearly
"""
From our 'employee_data.csv', analyze the salary distribution for employees 
who joined in the last 2 years. Provide the 25th, 50th, and 75th percentiles, 
and limit your response to a maximum of 4 bullet points.
"""
```



```python
# Bad: Lacks specific constraints
"""
Analyze the salary distribution from the employee data.
"""
```

#### Non-Data Example:



```python
# Good: Specifies constraints clearly
"""
Explain the concept of recursion in programming. Use exactly 3 sentences 
and provide a simple example that doesn't involve calculating factorials 
or Fibonacci numbers.
"""
```



```python
# Bad: Lacks specific constraints
"""
Explain recursion in programming.
"""
```

Remember: By specifying constraints such as time periods, specific metrics, response format, or content limitations, you can ensure that the response meets your needs exactly. This is crucial whether you're analyzing data from a CSV file or seeking general information on a topic.

### Practice (10 minutes)

Imagine you're asking for recommendations for a book to read.

1. Write a prompt without specifying any constraints.
2. Rewrite the prompt to include specific constraints. Constraints could include but not limited to:

    - Genre (e.g., science fiction, mystery, historical fiction)
    - Publication date (e.g., published in the last 5 years)
    - Awards or recognition (e.g., has won at least one major literary award)
    - Themes or topics (e.g., features themes of artificial intelligence or space exploration)
    - Target audience (e.g., suitable for young adult readers)
    - Length (e.g., under 400 pages long)
    - Format of the recommendation (e.g., provide title, author, and a brief description)
3. Show the class what you wrote for both prompts and explain the differences in the two model responses.

Consider including constraints such as:

- Genre (e.g., science fiction, mystery, historical fiction)
- Publication date (e.g., published in the last 5 years)
- Awards or recognition (e.g., has won at least one major literary award)
- Themes or topics (e.g., features themes of artificial intelligence or space exploration)
- Target audience (e.g., suitable for young adult readers)
- Length (e.g., under 400 pages long)
- Format of the recommendation (e.g., provide title, author, and a brief description)


### 4. Specifying Output Format

When working with data or seeking information, you can guide the model to provide responses in a specific format. This is particularly useful when you need structured outputs for further processing or analysis. This structured output can be in the form of a JSON, a table, a list, etc. Your example for this structure can either be a partial example or a full example.

##### Data Example:

#### Prompt without format specification:

```python
"""
Analyze the top 3 selling products from our 'annual_sales.csv' file.
"""
```

#### Possible Response:

```
Based on the analysis of the 'annual_sales.csv' file, the top 3 selling products are:

1. Smartphone X: This product led in units sold with 50,000 units, generating a revenue of $25,000,000.

2. Laptop Pro: While selling fewer units (30,000) than the Smartphone X, it generated the highest revenue at $45,000,000 due to its higher price point.

3. Wireless Earbuds: This product showed strong unit sales at 100,000, but had a lower revenue of $15,000,000, indicating a lower price point.

These top products demonstrate a mix of high-volume and high-value items, with smartphones leading in units sold and laptops in revenue generation.
```

To ensure a specific JSON format, you can provide the beginning of the JSON structure:

#### Prompt with format specification (using partial JSON structure):

```python
prompt = """
Analyze the top 3 selling products from our 'annual_sales.csv' file. 
Provide the output in the following JSON format:

{
  "top_products": [
    {
      "rank": 1,
      "name": 
"""
```

#### Possible Response:

```json
{
  "top_products": [
    {
      "rank": 1,
      "name": "Smartphone X",
      "units_sold": 50000,
      "revenue": 25000000
    },
    {
      "rank": 2,
      "name": "Laptop Pro",
      "units_sold": 30000,
      "revenue": 45000000
    },
    {
      "rank": 3,
      "name": "Wireless Earbuds",
      "units_sold": 100000,
      "revenue": 15000000
    }
  ],
  "analysis": "The top 3 products demonstrate a balance between high-volume and high-value items. Smartphone X leads in units sold, while Laptop Pro generates the highest revenue. Wireless Earbuds show strong unit sales but lower per-unit revenue."
}
```

## Non-Data Example:

#### Prompt without format specification:

```python
prompt = """
Describe the key features of Python programming language.
"""
```

#### Possible Response:

```
Python is a popular programming language known for several key features:

1. Easy to learn and read syntax: Python uses indentation and simple keywords, making it accessible for beginners and easy to read for all developers.

2. Interpreted language: Python code is executed line by line, allowing for rapid development and debugging.

3. Dynamically typed: Variables in Python can change types, providing flexibility in programming.

4. Object-oriented programming support: Python supports OOP concepts like classes, inheritance, and encapsulation.

5. Extensive standard library: Python comes with a large library of pre-built modules, reducing the need for external dependencies.

6. Cross-platform compatibility: Python programs can run on various operating systems with minimal modifications.

Python is widely used in web development, data analysis, artificial intelligence, machine learning, scientific computing, and automation tasks. The latest stable version as of May 2021 is 3.9.5.
```

To ensure a specific JSON format, you can provide the beginning of the JSON structure:

#### Prompt with format specification (using partial JSON structure):

```python
prompt = """
Describe the key features of Python programming language. 
Provide the output in the following JSON format:

{
  "language": "Python",
  "features": [
    {
      "name": 
"""
```

#### Possible Response:

```json
{
  "language": "Python",
  "features": [
    {
      "name": "Easy to learn and read syntax",
      "description": "Python uses indentation and simple keywords, making it accessible for beginners and easy to read for all developers."
    },
    {
      "name": "Interpreted language",
      "description": "Python code is executed line by line, allowing for rapid development and debugging."
    },
    {
      "name": "Dynamically typed",
      "description": "Variables in Python can change types, providing flexibility in programming."
    },
    {
      "name": "Extensive standard library",
      "description": "Python comes with a large library of pre-built modules, reducing the need for external dependencies."
    },
    {
      "name": "Cross-platform compatibility",
      "description": "Python programs can run on various operating systems with minimal modifications."
    }
  ],
  "version": "3.9.5",
  "popular_applications": ["Web development", "Data analysis", "AI and machine learning", "Scientific computing"]
}
```

Remember: By providing a structure for the desired output format, you can guide the model to generate responses in a consistent and easily parseable format, whether you're analyzing data from a CSV file or seeking information on programming concepts.

### Practice (10 minutes)

Imagine you're asking for information about a historical event.

1. Write a prompt without specifying any output format.
2. Rewrite the prompt to include a specific output format. Possible formats could include but not limited to:

- Bulleted list
- JSON
- Timeline
- Table
- Pros and cons list
- Q&A format
- Step-by-step guide
- Comparison chart
- Brief paragraph summary

Historical events could include:

- World War II
- The Great Fire of London
- The French Revolution
- The American Revolution  

3. Show the class what you wrote for both prompts and explain the differences in the two model responses.

In [ ]:
# comment and uncomment your prompts one at a time to see the difference
# Your prompt without specifying output format:
prompt = """
"""
# Your prompt with specifying output format:
prompt = """
"""

response = get_completion(
    prompt
)

print(response) # this is the response from the model

### 5. Including Few-Shot Examples in Your Queries

When working with data or seeking information, providing examples in your prompt can help guide the model to produce more accurate and appropriately formatted responses. This technique is called "few-shot learning" and can be applied to both data analysis tasks and general inquiries.

#### Definitions:

- **Zero-shot prompt**: A prompt that provides no examples, relying solely on instructions.
- **One-shot prompt**: A prompt that includes a single example to guide the model's response.
- **Few-shot prompt**: A prompt that contains multiple examples to demonstrate the desired pattern or format.

#### Data Example:

##### Zero-shot prompt:

```python
prompt = """
Analyze the sales data from our 'quarterly_sales.csv' file and provide a summary 
of the top-performing products.
"""
```

##### Few-shot prompt:

```python
prompt = """
Here are two examples of how to analyze and summarize sales data:

Example 1:
Input: Analyze Q1 sales for electronic products
Output:
1. Top product: Smartphones (Revenue: $1.2M, Units sold: 5000)
2. Fastest growing: Wireless earbuds (300% growth from last quarter)
3. Underperformer: Tablets (20% decline in sales)

Example 2:
Input: Summarize Q2 sales for home appliances
Output:
1. Best seller: Robot vacuums (Revenue: $800K, Units sold: 2000)
2. Surprise hit: Smart thermostats (150% increase in demand)
3. Needs attention: Microwave ovens (10% drop in market share)

Now, using the same format, analyze the sales data from our 'quarterly_sales.csv' 
file and provide a summary of the top-performing products for Q3.
"""
```

#### Non-Data Example:

##### One-shot prompt:

```python
prompt = """
Here's an example of how to explain a programming concept concisely:

Concept: List comprehension in Python
Explanation: A compact way to create lists based on existing lists or iterables, 
combining a for loop and a conditional statement in a single line of code.

Now, using the same concise format, explain the concept of 'decorator' in Python.
"""
```

##### Few-shot prompt:

```python
prompt = """
Here are two examples of explaining programming concepts:

Concept: List comprehension in Python
Explanation: A compact way to create lists based on existing lists or iterables, 
combining a for loop and a conditional statement in a single line of code.

Concept: Lambda functions in Python
Explanation: Anonymous, inline functions defined using the 'lambda' keyword, 
typically used for simple operations and as arguments to higher-order functions.

Now, using the same concise format, explain the following two concepts:
1. Decorator in Python
2. Generator expression in Python
"""
```

Remember: Including examples in your prompts can significantly improve the relevance and format of the responses, whether you're analyzing data from a CSV file or seeking explanations for programming concepts.

### Practice (15 minutes)

Your goal is to create prompts that will instruct a language model to convert informal text messages into more formal, professional language.

1. Write a zero-shot prompt for this task without providing any examples.
2. Write a few-shot prompt that includes at least two examples to guide the model's response. Possible examples could include but not limited to:

- Informal to Formal
- Short to Long
- Casual to Professional
- Friendly to Formal
- Short to Long

3. Show the class what you wrote for both prompts and explain the differences in the two model responses.

In [ ]:
# comment and uncomment your prompts one at a time to see the difference
# Your zero-shot prompt:
prompt = """
"""
# Your few-shot prompt:
prompt = """
"""

response = get_completion(
    prompt
)

print(response) # this is the response from the model   

### 6. System Instructions: Shaping AI Behavior

System instructions, also known as system prompts, are high-level directives that set the overall behavior and role of the AI model. They are powerful tools for defining how an AI model should behave and respond. These instructions can be used to continuously shape the behavior of the AI model without repeating the same instructions over and over again.

#### Data Example:

```python
# Bad System Instructions:
system_instructions = """
You're a data person. Look at sales numbers and tell me what's going on.
"""
```



```python
# Good System Instructions:
system_instructions = """
You are a data analysis expert specializing in sales trends. Your role is to analyze 
sales data and provide insights. Follow these guidelines:

1. Always use precise numerical values when reporting figures.
2. Provide a summary of overall trends before diving into specifics.
3. Highlight any anomalies or unexpected patterns in the data.
4. When comparing periods, use percentage changes for clarity.
5. Suggest possible reasons for significant changes in sales patterns.
6. Always specify the time period for which the analysis is being conducted.
7. If data is missing or seems inconsistent, flag it in your analysis.
8. Provide actionable insights based on the data whenever possible.
9. Use appropriate data visualization techniques when helpful (describe them in words).
10. If asked about future predictions, clearly state that these are projections based on current data.
"""
```

#### User Prompt:
```python
prompt = """
Analyze the quarterly sales data for our electronics department over the past year. 
Focus on identifying our best-selling products and any seasonal trends.
"""

```

#### Non-Data Example:



```python
# Bad System Instructions:
system_instructions = """
You know about writing. Help people write better.
""" 
```

```python
# Good System Instructions:
system_instructions = """
You are an expert in creative writing techniques. Your role is to guide aspiring 
writers in improving their craft. Adhere to these guidelines:

1. Provide constructive feedback that is both encouraging and honest.
2. Offer specific examples to illustrate writing principles.
3. Tailor advice to the genre or style the writer is working in.
4. Encourage experimentation with different techniques and styles.
5. When critiquing, always start with positive aspects before suggesting improvements.
6. Provide exercises or prompts to help writers practice specific skills.
7. Explain the reasoning behind your suggestions.
8. Respect the writer's unique voice and style while offering improvement ideas.
9. Recommend relevant books or resources for further learning when appropriate.
10. If asked about publishing or the business side of writing, provide current, 
    factual information or direct to reliable sources.
"""
```

#### User Prompt:
```python
prompt = """
I'm working on a short story and I'm struggling with dialogue. Can you give me some tips on how to make my characters' conversations sound more natural?
"""
```
Remember: System instructions set the overall context and behavior for the AI model, while user prompts are the specific queries or tasks within that context. This separation allows for consistent behavior across multiple interactions without repeating the same instructions.

### Practice (20 minutes)

Your task is to create system instructions for an AI language model that will act as a basic calculator assistant.

1. Write a set of system instructions that define the AI's role as a calculator. Keep it simple and focused on basic arithmetic operations (addition, subtraction, multiplication, division).

2. Create two example user prompts that someone might ask this calculator assistant. For example:
   - A simple arithmetic question
   - A request to explain a basic math concept
When creating your system instructions for the basic calculator assistant, you might want to think about:

- How to handle decimal places in calculations
- Whether to show step-by-step solutions
- How to respond to requests for operations beyond basic arithmetic
- Dealing with unclear or ambiguous requests
- Explaining mathematical concepts in addition to performing calculations
- Using consistent mathematical notation
- Handling potential division by zero errors
- Whether to accept and understand various input formats (e.g., fractions, percentages)
- How to respond if asked about the history or cultural context of mathematical concepts
- Balancing brevity with clarity in responses

Remember, you don't need to address all of these points in your instructions. Choose the ones you think are most important for a basic calculator assistant.

3. Share your system instructions and example user prompts with the class. Briefly explain how your instructions shape the AI's behavior.

Tips:
- Keep the system instructions concise, aiming for 5-7 clear guidelines.
- Focus on accuracy and clarity in mathematical operations.
- Consider how to handle potential errors or unclear requests.




In [ ]:
def get_completion(prompt,system_instruction=None, model="gemini-1.5-flash", **kwargs):
    model = genai.GenerativeModel(model)
    
    if system_instruction:
        prompt = f"{system_instruction}\n\n{prompt}"
    
    # Create a generation_config dictionary with default values
    generation_config = {
        "temperature": 0.7,
        "min_length": 20,
        "max_length": 200,
        "max_output_tokens": 2020,
    }
    
    # Update generation_config with any provided kwargs
    generation_config.update(kwargs)
    
    response = model.generate_content(prompt, generation_config=generation_config)
    return response.text

In [ ]:
# comment and uncomment your prompts one at a time to see the difference
# Your basic system instruction:
system_instruction = """
"""
# Your more effective and specialized system instruction:
system_instruction = """
"""                         

